[View in Colaboratory](https://colab.research.google.com/github/iu-data-science-python-i590/final-project-team-python-trio/blob/master/Phase1/final_project_phase1.ipynb)

## Analysis of K-means Clustering on Wisconsin Breast Cancer Data

Phase 1.0 | 2018 October 24

Bill Screen, Ha-Lan Nguyen, Tarun Rawat | Indiana University | M.S. Data Science

#### PROBLEM STATEMENT: 
Breast cancer is a rising issue among women. A cancer’s stage is a crucial factor in deciding what treatment options to recommend, and in determining the patient’s prognosis. Today, in the United States, approximately one in eight women over their lifetime has a risk of developing breast cancer. An analysis of the most recent data has shown that the survival rate is 88% after 5 years of diagnosis and 80% after 10 years of diagnosis. With early detection and treatment, it is possible that this type of cancer will go into remission. In such a case, the worse fear of a cancer patient is the recurrence of the cancer.

#### OBJECTIVE: 
This report will demonstrate how implementing a "k-means" algorithm can be used to classify benign and malign cells in two different groups.

In [0]:
print(''.join(map(str,range(10))))

0123456789
